In [5]:
%load_ext autoreload
%autoreload 2

import sys
import os
import inspect
import getpass
import json

import pandas as pd

project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../../" )
sys.path.insert(0, project_path)
print(project_path)

from nestor.store_data import helper

from nestor.store_data.database import DatabaseNeo4J 
from nestor.store_data import integration 

from nestor.store_data.objects import human
from nestor.store_data.objects import issue
from nestor.store_data.objects import machine
from nestor.store_data.objects import tag
from nestor.store_data.objects import maintenanceworkorder


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/sam11/Git/nestor


# Setup Dataset informations

foreach Dataset get all the specific information such as:

    bolt : the url of the database server {e.g.: bolt://localhost:7687}
    user : Identification to connect to the database -STRING
    password : Password to connect to the database -STRING
    
    nestor_file: the h5 files generated by Nestor (the tagging app)
    csvSchemas_file : Path of the file use to link the header of the CSV with the properties in the database -YAML
    
------------------------------------
    
    vocab1g_file : Path of the vocab 1G file created by Nestor -CSV
    vocabNg_file : Path of the vocab NG file created by Nestor -JSON
    
    tree_path : Path of the item-item hierarchy (created by <insert a name here>) -JSON

In [6]:
bolt = input('bolt:')
user = input('user:')
password = getpass.getpass("Password:")

nestor_file = input('path of the nestor file:')
csvSchemas_file = input('path of the csv header links:')

bolt:k;
user:k,
Password:········
path of the nestor file:b
path of the csv header links:n


In [7]:
bolt = "bolt://localhost:7687"
user = "neo4j"
password = "MINE!!"

nestor_file = os.path.join(project_path ,'data','public', 'output.h5')
csvSchemas_file = os.path.join(project_path ,'data','public', 'header.yaml')

# Load Files from Path

foreach path file, load them into the memory using the needed format:
    
    Dataframe for : csv_file, bincsv_file, binNcsv_file, vocabcsv_file, vocabNcsv_file
   
    Dictionary for : csvSchemas_file, tree_path

In [8]:
dataframe = pd.read_hdf(nestor_file, key="df")
dataframe.fillna("", inplace=True)
csvSchemas_dict = helper.openYAMLFile(csvSchemas_file)


dataframe_tag = pd.read_hdf(nestor_file, key="tags")
dataframe_rel = pd.read_hdf(nestor_file, key="rels")


dataframe_vocab1g = None
dataframe_vocabNg = None

/Users/sam11/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


yaml file open


In [ ]:
with open(tree_path) as file:
    tree_dict = json.load(file)

# ConnectGraphDatabase

In [9]:
databaseSchema_file = os.path.join(project_path ,'nestor', 'store_data', 'DatabaseSchema.yaml')
databaseSchema_dict = helper.openYAMLFile(databaseSchema_file)

database = DatabaseNeo4J(bolt, user, password, databaseSchema_dict) 

yaml file open


## Clean the database

In [10]:
database.deleteData()

database.dropConstraints()
database.dropIndexes()

database.createIndexes()
database.createConstraints()

1

## Create Cypher Queries

### Create historical_data

In [11]:
database.runQueries(integration.cypherCreate_historicalMaintenanceWorkOrder (database.schema, dataframe, csvSchemas_dict))

100%|██████████| 3438/3438 [01:27<00:00, 39.47it/s]


1

### Create tag_data

In [12]:
database.runQueries(integration.cypherCreate_tag(database.schema, dataframe_tag, dataframe_vocab1g, dataframe_vocabNg))

100%|██████████| 815/815 [00:13<00:00, 59.00it/s]


1

In [13]:
database.runQueries(integration.cypherCreate_tag(database.schema, dataframe_rel, dataframe_vocab1g, dataframe_vocabNg))

100%|██████████| 1032/1032 [00:10<00:00, 98.99it/s]


1

### Update 1gram-->Ngram

In [14]:
database.runQueries(integration.cypherLink_Ngram1gram(database.schema))

100%|██████████| 6/6 [00:00<00:00, 18.01it/s]


1

### Update issue-->item

In [15]:
database.runQueries(integration.cypherLink_itemIssue(database.schema))

100%|██████████| 2/2 [00:00<00:00,  4.51it/s]


1

### Update item-->item

In [ ]:
database.runQueries(integration.cypherCreate_itemsTree(database.schema,tree_dict ))

------------------